<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-03 10:22:36
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.37 C
-------------------
Today PnL: 34.91 K (0.26%)
Current PnL: -13.25 L (-9.23%)
CY Booked + Current PnL: -6.51 L (-4.54%)
-------------------
Total profit:  4.72 L
Total loss:  -17.97 L
-------------------
Total Booked + Current PnL: 20.47 L (17.57%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.93%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.68 L (52.46%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,HINDALCO,651.95,761.55,-18.25,H-LC,89.58,113200.0,8888.0,8648.0,1.31,8.52,7.64,16.81,27.0,1.03,0.83,25.93,X5K,ATH,METALS
87,UJJIVANSFB,52.77,53.00,54.03,M-SC,93.75,134028.0,-8451.0,9074.0,-0.52,-5.93,6.77,0.44,243.0,-0.93,0.98,58.19,OX40N,NTT,BANKS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,42.71,102709.0,24347.0,9685.0,0.19,31.07,9.43,43.43,77.0,2.51,0.75,38.04,XY24,NTT,MISC
5,APOLLOHOSP,6904.43,8285.00,-17.07,H-LC,86.46,112965.0,9399.0,11308.0,1.21,9.07,10.01,20.00,30.0,0.83,0.83,24.43,X40N,BTT,HEALTHCARE
88,UNIONBANK,123.87,163.00,0.15,M-LC,36.46,173267.0,32427.0,12059.0,-0.65,23.02,6.96,31.59,89.0,2.69,1.27,50.51,XY24,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-30.08,L-SC,100.00,140858.0,-984.0,57555.0,-1.06,-0.69,40.86,39.88,259.0,-0.02,1.03,42.75,XY24,NTT,POWER
38,HEROMOTOCO,4311.81,6039.03,-6.67,H-MC,45.83,149835.0,-1078.0,61537.0,0.88,-0.71,41.07,40.06,102.0,-0.02,1.10,22.56,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,HONAUT,42646.15,58357.33,-15.73,X-MC,56.25,121440.0,-6498.0,53628.0,4.29,-5.08,44.16,36.84,56.0,-0.12,0.89,24.32,X40N,ATH,ELECTRICAL
15,BLUESTARCO,1646.70,2337.55,-2.67,H-SC,97.92,182550.0,17880.0,51205.0,3.95,10.86,28.05,41.95,121.0,0.35,1.34,19.13,X40N,ATH,AC
93,VOLTAS,1278.28,1918.49,-24.94,H-MC,87.50,204375.0,12633.0,83405.0,2.23,6.59,40.81,50.08,100.0,0.15,1.50,13.83,XY25,ATH,AC
6,ASIANPAINT,2961.56,4250.00,-15.98,H-LC,84.38,166974.0,-34412.0,122025.0,1.49,-17.09,73.08,43.51,32.0,-0.28,1.22,16.23,X40,BTT,PAINTS
36,HAVELLS,1588.02,2069.16,-5.03,X-MC,55.21,245357.0,-2374.0,77435.0,1.44,-0.96,31.56,30.30,53.0,-0.03,1.80,12.21,X40,ATH,ELECTRICAL


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
30,DMART,3764.92,5201.00,-17.03,X-LC,57.29,208407.0,23926.0,46433.0,-3.16,12.97,22.28,38.14,15.0,0.52,1.53,24.95,X40N,NTT,FMCG
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16458.0,-33135.0,2059.0,-1.99,-66.81,12.51,-62.66,NaN,-16.09,0.12,10.37,XY24,ATH,MISC
44,ICICIPRULI,600.83,790.00,-9.12,H-MC,43.75,146844.0,11056.0,31704.0,-1.37,8.14,21.59,31.48,119.0,0.35,1.08,21.25,X40,ATH,INSURANCE
85,TRIDENT,37.35,48.00,20.73,M-SC,31.25,75903.0,-16352.0,42657.0,-1.25,-17.72,56.20,28.51,230.0,-0.38,0.56,29.88,XR,NTT,TEXTILES
31,EASEMYTRIP,18.12,26.40,5.52,M-SC,9.38,101581.0,-75234.0,156028.0,-1.23,-42.55,153.60,45.70,190.0,-0.48,0.74,0.39,XY24,NTT,TRAVEL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,54.03,M-SC,93.75,134028.0,-8451.0,9074.0,-0.52,-5.93,6.77,0.44,243.0,-0.93,0.98,58.19,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-214.35,H-SC,96.88,82446.0,-12870.0,12911.0,0.67,-13.50,15.66,0.04,154.0,-1.00,0.60,62.62,OX40N,NTT,MISC
46,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,91.67,149197.0,-25362.0,25393.0,0.80,-14.53,17.02,0.02,127.0,-1.00,1.09,29.38,OX40N,NTT,PAINTS
79,SYMPHONY,1306.42,1306.0,-31.65,M-SC,6.25,145567.0,-25574.0,25518.0,0.41,-14.94,17.53,-0.03,189.0,-1.00,1.07,4.62,OX40N,NTT,DURABLES
81,TATAELXSI,7332.28,7332.0,-17.14,X-MC,18.75,86954.0,-15698.0,15695.0,0.42,-15.29,18.05,-0.00,58.0,-1.00,0.64,32.18,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,8.33,48162.0,6907.0,3896.0,0.81,16.74,8.09,26.19,1.0,1.77,0.35,13.39,XY24,ATH,IT
83,TCS,3794.03,4998.00,-18.06,X-LC,40.62,266612.0,-29322.0,123228.0,-0.15,-9.91,46.22,31.73,1.0,-0.24,1.95,6.71,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,61.46,288280.0,26679.0,118944.0,0.23,10.20,41.26,55.67,2.0,0.22,2.11,16.93,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.36,X-LC,28.12,258720.0,-11627.0,46259.0,0.13,-4.30,17.88,12.81,6.0,-0.25,1.89,8.20,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-11.46,X-LC,34.38,280695.0,15269.0,41627.0,0.46,5.75,14.83,21.44,9.0,0.37,2.06,12.31,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,0.15,M-LC,36.46,173267.0,32427.0,12059.0,-0.65,23.02,6.96,31.59,89.0,2.69,1.27,50.51,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,42.71,102709.0,24347.0,9685.0,0.19,31.07,9.43,43.43,77.0,2.51,0.75,38.04,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,70.83,72925.0,38476.0,1473.0,-1.16,111.69,2.02,115.97,NaN,26.12,0.53,11.51,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,98.52,M-MC,68.75,237774.0,12812.0,154838.0,0.44,5.70,65.12,74.53,185.0,0.08,1.74,38.05,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,27.10,H-SC,72.92,130082.0,4785.0,54426.0,0.27,3.82,41.84,47.25,139.0,0.09,0.95,62.74,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.09,H-SC,41.67,119084.0,17080.0,27949.0,0.52,16.74,23.47,44.14,124.0,0.61,0.87,35.71,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,42.71,102709.0,24347.0,9685.0,0.19,31.07,9.43,43.43,77.0,2.51,0.75,38.04,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,36.46,173267.0,32427.0,12059.0,-0.65,23.02,6.96,31.59,89.0,2.69,1.27,50.51,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,70.83,72925.0,38476.0,1473.0,-1.16,111.69,2.02,115.97,NaN,26.12,0.53,11.51,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.18,M-SC,90.62,203992.0,32371.0,129698.0,0.05,18.86,63.58,94.43,198.0,0.25,1.49,50.56,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.41,M-LC,81.25,166904.0,15959.0,93500.0,0.82,10.57,56.02,72.51,69.0,0.17,1.22,14.97,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-48.08,H-SC,80.21,135388.0,12052.0,122783.0,-0.58,9.77,90.69,109.32,122.0,0.10,0.99,33.75,AR,ATH,MISC
60,NATIONALUM,189.63,247.44,-48.64,H-MC,71.88,103345.0,2841.0,27800.0,1.34,2.83,26.90,30.49,97.0,0.10,0.76,36.15,MH,ATH,METALS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,68160.0,-45389.0,85439.0,-0.06,-39.97,125.35,35.27,261.0,-0.53,0.50,66.82,XR,NTT,HOTELS
58,MASFIN,326.60,402.39,-19.15,H-SC,58.33,91890.0,-6090.0,28826.0,-0.44,-6.22,31.37,23.21,137.0,-0.21,0.67,33.27,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,8.10,X-SC,98.96,100642.0,-321.0,94855.0,-0.26,-0.32,94.25,93.63,64.0,-0.00,0.74,49.90,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,42.71,102709.0,24347.0,9685.0,0.19,31.07,9.43,43.43,77.0,2.51,0.75,38.04,XY24,NTT,MISC
90,VALIANTORG,512.64,838.00,-355.77,H-SC,5.21,108680.0,-24606.0,109202.0,0.40,-18.46,100.48,63.47,148.0,-0.23,0.80,82.17,XR,NTT,CHEMICALS


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-30.08,L-SC,100.00,140858.0,-984.0,57555.0,-1.06,-0.69,40.86,39.88,259.0,-0.02,1.03,42.75,XY24,NTT,POWER
4,ALKYLAMINE,2347.98,4546.37,8.10,X-SC,98.96,100642.0,-321.0,94855.0,-0.26,-0.32,94.25,93.63,64.0,-0.00,0.74,49.90,SR,ATH,CHEMICALS
15,BLUESTARCO,1646.70,2337.55,-2.67,H-SC,97.92,182550.0,17880.0,51205.0,3.95,10.86,28.05,41.95,121.0,0.35,1.34,19.13,X40N,ATH,AC
92,VIPIND,488.80,489.00,-214.35,H-SC,96.88,82446.0,-12870.0,12911.0,0.67,-13.50,15.66,0.04,154.0,-1.00,0.60,62.62,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-53.72,L-SC,95.83,68160.0,-45389.0,85439.0,-0.06,-39.97,125.35,35.27,261.0,-0.53,0.50,66.82,XR,NTT,HOTELS


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.92
1,25,41.94
2,50,70.45


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.41
LC    34.00
MC    21.61
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.53
X40      12.25
XY25     12.03
X40N     11.86
XR       10.10
AR        8.21
OX40N     8.21
MH        1.71
X5K       1.49
X200      1.40
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.65
M-SC    15.94
H-LC    14.18
X-LC    12.12
H-MC     9.55
X-MC     7.58
M-LC     6.65
X-SC     4.39
M-MC     4.12
L-SC     2.43
L-LC     1.05
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.05,-1.79,35.84
IT,12.11,-9.79,63.71
BANKS,7.90,-5.60,48.69
FINANCE,7.88,-12.14,52.73
MISC,6.61,-14.98,68.33
PAINTS,5.74,-11.17,33.08
ELECTRICAL,5.52,2.21,43.47
HEALTHCARE,4.53,-4.43,34.93
AUTO,4.33,-14.30,64.56


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2695588.0,27
XR,1079489.0,13
AR,1023447.0,10
X40,647170.0,9
X40N,545327.0,11
XY25,475597.0,8
OX40N,309595.0,11
SR,176198.0,2
MH,82226.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1964385.0,20
M-SC,1766428.0,21
H-LC,604506.0,12
H-MC,531723.0,8
X-LC,520009.0,7
X-MC,417579.0,6
M-MC,406380.0,3
X-SC,320453.0,5
L-SC,264946.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      864141.0      6
M-SC       XY24      817217.0      7
H-SC       AR        524186.0      3
M-SC       XR        334686.0      4
H-MC       XY24      297382.0      3
M-MC       XY24      296454.0      2
X-LC       X40       242172.0      2
H-SC       XR        217516.0      3
           X40N      217122.0      4
H-LC       X40N      198249.0      4
M-SC       AR        181505.0      2
           OX40N     165824.0      5
M-LC       XY24      157535.0      3
H-LC       X40       152589.0      3
X-MC       X40       149121.0      2
L-SC       XR        147598.0      2
X-SC       XR        122783.0      1
X-LC       AR        116562.0      1
           XY25      114842.0      3
M-SC       XY25      114269.0      1
M-MC       XR        109926.0      1
X-MC       XY25      103443.0      1
           XY24       95692.0      1
X-SC       SR         94855.0      1
M-LC       XR         93500.0      1
H-SC       OX40N      86994.0      3
H-MC       XY25       83405.0      1
M-SC       SR         81343.0      1
H-LC       AR         78391.0      1
M-SC       X40        71584.0      1
H-LC       X200       68463.0      1
           X5K        64890.0      2
X-SC       XY24       61733.0      1
H-MC       AR         61537.0      1
L-SC       AR         59793.0      1
           XY24       57555.0      1
H-SC       MH         54426.0      1
X-MC       X40N       53628.0      1
L-MC       XR         53480.0      1
X-LC       X40N       46433.0      1
L-LC       XY25       45862.0      1
H-LC       XY24       41924.0      1
X-SC       OX40N      41082.0      2
H-MC       X40        31704.0      1
           X40N       29895.0      1
           MH         27800.0      1
X-MC       OX40N      15695.0      1
M-LC       XY25       13776.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 46.0 seconds
